In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23021432
paper_name = 'orij_smits_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/pH Screen raw.xlsx', sheet_name='initial screens')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5378 x 4


In [7]:
original_data.head()

,ORF,Gene,pH_cytosolic,pH_cytosolic_2
0,YOR202W,HIS3,6.927679,7.019926
1,YOR202W,HIS3,6.920939,6.991633
2,YOR202W,HIS3,6.917576,6.944440
3,YOR202W,HIS3,6.933141,7.085373
4,YOR202W,HIS3,6.942147,7.150766


In [9]:
original_data['orf'] = original_data['ORF '].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            ORF  Gene  pH_cytosolic  pH_cytosolic_2  orf
index_input                                             
1068         NaN  NaN           NaN             NaN  NAN
1069         NaN  NaN           NaN             NaN  NAN


In [13]:
original_data = original_data.loc[t,:]

In [15]:
original_data['data'] = original_data[['pH_cytosolic','pH_cytosolic_2']].mean(axis=1)

/opt/anaconda3/envs/yp-data/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(4268, 1)

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [126]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,126
data_type,value
orf,
YAL002W,6.837413
YAL004W,6.946167
YAL005C,7.068710
YAL007C,6.889098
YAL008W,7.054038


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,126
,data_type,value
gene_id,orf,
2,YAL002W,6.837413
1863,YAL004W,6.946167
4,YAL005C,7.068710
5,YAL007C,6.889098
6,YAL008W,7.054038


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            126          
data_type           value    valuez
gene_id orf                        
2       YAL002W  6.837413 -0.015020
1863    YAL004W  6.946167 -0.000328
4       YAL005C  7.068710  0.016226
5       YAL007C  6.889098 -0.008038
6       YAL008W  7.054038  0.014244

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 23021432...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.08s/it]

Updating the data_modified_on field...
